<h1>F20BC Coursework</h1>
<p style="font-size:15px;">This notebook implements and analyses the Biologically-Inspired Computation coursework, focusing on Artificial Neural Networks (ANNs) and Particle Swarm Optimization (PSO). It includes a multi-layer ANN architecture trained using PSO to solve a regression task predicting concrete compressive strength. Additionally, it explores the impact of hyperparameters on model performance through systematic experiments.</p>

**<p style="font-size:18px;">Data Preparation</p>**
This contains all the data preprocessing and cleaning</p>

In [26]:
import os
print(os.getcwd())

/content


In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [28]:
df = pd.read_csv("concrete_data.csv", skiprows=1)

In [29]:
df.head()

,cement,blast_furnace_slag,fly_ash,water,superplasticizer,coarse_aggregate,fine_aggregate,age,concrete_compressive_strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [30]:
# Viewing the dimensionality of the dataset
df.shape

(1030, 9)

In [31]:
df.dropna(inplace = True)
df.isnull().sum()

,0
cement,0
blast_furnace_slag,0
fly_ash,0
water,0
superplasticizer,0
coarse_aggregate,0
fine_aggregate,0
age,0
concrete_compressive_strength,0


In [32]:
print(f'Duplicate rows: {df.duplicated().sum()}')

Duplicate rows: 25


In [33]:
# Remove duplicate rows
df.drop_duplicates(inplace=True)

**<p style="font-size:18px;">Scaling and Encoding</p>**

In [34]:
print(df.head())
print(df.columns)

   cement  blast_furnace_slag  fly_ash  water  superplasticizer  \
0   540.0                 0.0      0.0  162.0               2.5   
1   540.0                 0.0      0.0  162.0               2.5   
2   332.5               142.5      0.0  228.0               0.0   
3   332.5               142.5      0.0  228.0               0.0   
4   198.6               132.4      0.0  192.0               0.0   

   coarse_aggregate  fine_aggregate   age  concrete_compressive_strength  
0            1040.0            676.0   28                          79.99  
1            1055.0            676.0   28                          61.89  
2             932.0            594.0  270                          40.27  
3             932.0            594.0  365                          41.05  
4             978.4            825.5  360                          44.30  
Index(['cement', 'blast_furnace_slag', 'fly_ash', 'water', 'superplasticizer',
       'coarse_aggregate', 'fine_aggregate ', 'age',
       'concret

In [35]:
from sklearn.model_selection import train_test_split

X = df.iloc[:, :-1] # All columns except the last
y = df.iloc[:, -1] # The last column

In [36]:
print(X.shape)
print(y.shape)

(1005, 8)
(1005,)


In [37]:
print(X.head())

   cement  blast_furnace_slag  fly_ash  water  superplasticizer  \
0   540.0                 0.0      0.0  162.0               2.5   
1   540.0                 0.0      0.0  162.0               2.5   
2   332.5               142.5      0.0  228.0               0.0   
3   332.5               142.5      0.0  228.0               0.0   
4   198.6               132.4      0.0  192.0               0.0   

   coarse_aggregate  fine_aggregate   age  
0            1040.0            676.0   28  
1            1055.0            676.0   28  
2             932.0            594.0  270  
3             932.0            594.0  365  
4             978.4            825.5  360  


In [38]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [39]:
print("First 5 rows of scaled training data:")
print(X_train_scaled[:5])

First 5 rows of scaled training data:
[[-0.7036142   0.7362629  -0.8810702   0.1799759  -1.0226942   1.32341753
  -0.17765612 -0.62455318]
 [-0.83112259 -0.84582676  1.04788182 -0.72981642  0.63596651  1.35948653
   0.32102131  0.88513966]
 [ 0.43140366  1.06620495 -0.8810702   0.38479825 -0.18498677 -1.33280675
   0.00745899 -0.28365479]
 [-1.05233032  0.66864226  1.10026551 -0.30588178  0.28412939  0.42298068
  -0.33758549 -0.51092038]
 [-1.17404287 -0.84582676  1.31904441  0.5419875   0.50193332 -1.24005789
   1.18741037 -0.28365479]]


**<p style="font-size:18px;">Artificial Neural Network (ANN)</p>**
Below are the implementations of the ANN to train and predict concrete compressive strength

In [40]:
class activationFunction:
    def logisticFunction(x):
        return 1 / (1 + np.exp(-x))

    def reluFunction(x):
        return np.maximum(0, x)

    def hyperbolicFunction(x):
        return np.tanh(x)

    def leakyReLU(x, alpha=0.01):
        return np.maximum(x, alpha * x)

    def linearFunction(x):  # Identity function for regression output
        return x

    # def elu(x, alpha=1.0):
    #     return np.where(x > 0, x, alpha * (np.exp(x) - 1))

class ArtificialNeuralNetwork:
    def __init__(self, layerSize, activationFunction):
        self.layerSize = layerSize
        self.activationFunction = activationFunction
        self.weights = [np.random.randn(layer_sizes[i], layer_sizes[i + 1]) * np.sqrt(2 / layer_sizes[i]) for i in range(len(layer_sizes) - 1)]
        self.biases = [np.random.randn(1, layer_sizes[i + 1]) for i in range(len(layer_sizes) - 1)]

    def visualize_layers(self):
        for i in range(len(self.layerSize)-1):
            print("LAYER COUNT:  ", i,"    NODES: ",self.layerSize[i])
            print(i)
            print(f"  Weights Shape:   ", self.weights[i].shape)
            print(f"  Biases Shape:    ",self.biases[i].shape )
            print(f"  Weights= : {self.weights[i]}")
            print(f"  Biases= {self.biases[i]}")

    def forwardPropagation(self, x):
        output = x
        for i in range(len(self.weights)):
            matrix_total = np.dot(output, self.weights[i]) + self.biases[i]
            output = self.activationFunction[i](matrix_total)
        return output

Layer size is structure to be 8,16,8,1 because
- 8 input layers
- 16 neurons in first hidden layer
- 8 for the second hidden layer
- 1 for the output layer<p>
This is to check the predicted values generated after forward propagation from the ANN.</p>

In [41]:
layer_sizes = [8, 16, 8, 1]

activation_functions = [
    activationFunction.logisticFunction,
    activationFunction.reluFunction,
    activationFunction.linearFunction,
]
ann = ArtificialNeuralNetwork(layer_sizes, activation_functions)
ann.visualize_layers()

x_example = np.random.rand(8, 8)  #8 samples, 8 features
output = ann.forwardPropagation(x_example)
print("ANN Output (Sample Predictions):", output)

LAYER COUNT:   0     NODES:  8
0
  Weights Shape:    (8, 16)
  Biases Shape:     (1, 16)
  Weights= : [[-0.16391351  0.27130922 -0.38130975 -0.89043275  0.27675003  0.40445118
   0.12725068 -0.32677117  0.28832943 -0.43567489 -0.61622054  0.40867587
   0.51253698 -0.82238288  0.09994806  0.41455674]
 [ 0.01357476  0.70744106  0.89711056 -0.20249589  0.03513424  0.33745879
   0.9160188  -0.10554073  0.19320343 -1.26798237 -0.01644423 -0.63896862
   0.25574349  0.43407544  1.14300521  0.23506576]
 [-0.6709218  -0.37709073 -1.47135305 -0.83820953  0.12680984 -0.28331022
   0.15741677  0.69973161  0.11213985 -0.39845932  0.15544696  0.59905448
   0.05136906  0.39972915 -0.59221603 -0.14131407]
 [ 0.27888381  0.12961833 -0.33610961 -0.3154581   0.21035131 -0.09957504
  -1.07117779 -0.63409355 -0.27500263  0.20151186  0.57331009  0.23025922
   0.11117629 -0.13590431  0.24284185 -1.03927827]
 [ 0.74539969 -0.12464923  0.60479678  0.00945138  0.50620615  0.83426845
  -0.9886836   0.00274827  0

**<p style="font-size:18px;">Loss Function</p>**
Since the problem domian is regression, MSE is utilised as the loss function

In [42]:
class lossFunction:
    def evaluate(self,y_pred,y_train):
        self.y_pred=y_pred
        self.y_train=y_train
class MeanSquaredError(lossFunction):
    def evaluate(self, y_pred, y_train):
        return np.mean((y_pred - y_train) ** 2)

In [43]:
y_train = y_train.to_numpy().reshape(-1, 1)
y_pred = ann.forwardPropagation(X_train_scaled) # Forward propagating  through the ANN
# checking y_pred and y_train has the correct shape
print("y_pred shape:", y_pred.shape)
print("y_train shape:", y_train.shape)

# use to calculate the loss function
loss_function=MeanSquaredError()
loss=loss_function.evaluate(y_pred, y_train)
print("Loss:", loss)

y_pred shape: (703, 1)
y_train shape: (703, 1)
Loss: 1486.0842941302046


**<p style="font-size:18px;">Implement PSO Algorithm</p>**

In [44]:
class Particle:
    def __init__(self,vectorSize):
            self.particlePosition=np.random.rand(vectorSize)
            self.particleVelocity=np.random.rand(vectorSize)
            self.bestPosition=np.copy(self.particlePosition)
            self.informants=[]

In [45]:
def particleToAnn(particle, annLayers, activationFunctions):
    neuralNetwork = ArtificialNeuralNetwork(layerSize=annLayers, activationFunction=activationFunctions)
    weightBiasIndexCount = 0
    for i in range(len(annLayers) - 1):
        prevValue = annLayers[i]
        nextValue = annLayers[i + 1]
        weightRange = prevValue * nextValue

        weight = particle.particlePosition[weightBiasIndexCount:weightBiasIndexCount + weightRange].reshape((prevValue, nextValue))
        weightBiasIndexCount += weightRange

        biases = particle.particlePosition[weightBiasIndexCount:weightBiasIndexCount + nextValue].reshape((1, nextValue))
        weightBiasIndexCount += nextValue

        # activation = activationFunctions[i]
        neuralNetwork.weights[i] = weight
        neuralNetwork.biases[i] = biases
    return neuralNetwork

In [46]:
def assessFitness(particle, dataset, annLayers, activationFunctions, loss_function):
    x, y = dataset
    ann = particleToAnn(particle, annLayers, activationFunctions)
    predictions = ann.forwardPropagation(x)
    loss = loss_function.evaluate(predictions, y.reshape(-1, 1))
    return loss

In [55]:
class ParticleSwarmOptimisation:
    def __init__(self, swarmSize, alpha, beta, delta, gamma, jumpSize, informantCount, vectorSize):
        self.swarmSize = swarmSize
        self.alpha = alpha
        self.beta = beta
        self.delta = delta
        self.gamma =gamma
        self.jumpSize = jumpSize
        self.informantCount = informantCount
        self.vectorSize = vectorSize
        self.global_best = None
        self.global_best_fitness = float('inf')

    def initInformants(self, informantCount, particleArray):
        for p in particleArray:
            potentialInformants = [potInf for potInf in particleArray if potInf != p]
            p.informants = [random.choice(potentialInformants) for _ in range(informantCount)]

    def get_best_informant(self, particle, dataset, annLayers, activationFunctions, loss_function):
        bestInf = None
        bestFitnessInf = float('-inf')
        for i in particle.informants:
            fitness = assessFitness(i, dataset, annLayers, activationFunctions, loss_function)
            if fitness > bestFitnessInf:
                bestFitnessInf = fitness
                bestInf = i
        return bestInf.particlePosition

    def psoOptimisation(self, swarmSize, alpha, beta, gamma, jumpSize, informantCount, vectorSize,
                        dataset, annLayers, activationFunctions, loss_function, max_iterations=100):

        particleArray = [Particle(vectorSize) for _ in range(swarmSize)]
        self.initInformants(informantCount, particleArray)

        best = None
        iteration = 0

        while iteration < max_iterations:
            # Update global best
            for p in particleArray:
                particleFitness = assessFitness(p, dataset, annLayers, activationFunctions, loss_function)
                if best is None or particleFitness < assessFitness(best, dataset, annLayers, activationFunctions, loss_function):
                    best = p

            for p in particleArray:
                previousBest = p.bestPosition
                informantsBest = self.get_best_informant(p, dataset, annLayers, activationFunctions, loss_function)
                allBest = best.bestPosition

                b = np.random.uniform(0.0, beta)
                c = np.random.uniform(0.0, gamma)
                d = np.random.uniform(0.0, delta)

                updatedVelocity = (
                    alpha * p.particleVelocity +
                    b * (previousBest - p.particlePosition) +
                    c * (informantsBest - p.particlePosition) +
                    d * (allBest - p.particlePosition)
                )

                p.particleVelocity = updatedVelocity
                p.particlePosition += jumpSize * updatedVelocity

            iteration += 1

        return best.particlePosition

<p style="font-size:18px;">Test out PSO Functionality</p>

In [57]:
# Code to test out PSO functionality
X_train_scaled = np.random.rand(100, 8)  # Simulated scaled training features (100 samples, 8 features)
y_train = np.random.rand(100, 1)  # Simulated training labels (100 samples, 1 output)

# define hyperparameters for PSO
swarmSize = 10
alpha = 0.5  # inertia weight
beta = 0.5   # cognitive parameter
gamma = 0.5  # social parameter
delta = 0.5
jumpSize = 0.5
informantCount = 3
annLayers=layer_sizes
vectorSize = sum([(annLayers[i] * annLayers[i + 1]) + annLayers[i + 1] for i in range(len(annLayers) - 1)])  # Total weights + biases
max_iterations = 50

# initialize the PSO optimiser
pso = ParticleSwarmOptimisation(
    swarmSize = swarmSize,
    alpha = alpha,
    beta = beta,
    delta = delta,
    gamma = gamma,
    jumpSize = jumpSize,
    informantCount = informantCount,
    vectorSize = vectorSize
)

# run it into the psoOptimisation
best_parameters = pso.psoOptimisation(
    swarmSize = swarmSize,
    alpha = alpha,
    beta = beta,
    gamma = delta,  # Using delta as gamma as per your code
    jumpSize = jumpSize,
    informantCount = informantCount,
    vectorSize = vectorSize,

    dataset = (X_train_scaled, y_train),
    annLayers = annLayers,
    activationFunctions = activation_functions,
    loss_function= MeanSquaredError(),
    max_iterations = max_iterations
)
print("Optimised ANN Parameters:", best_parameters)

Optimised ANN Parameters: [0.40036797 0.10195019 0.27178271 0.74647028 0.65379347 0.7303438
 0.15340326 0.63270038 0.52051898 0.29525368 0.58992008 0.57437225
 0.86073841 0.35942464 0.74961645 0.68021561 0.38887384 0.69568626
 0.05308896 0.6530167  0.69562674 0.30395803 0.21740237 0.74275927
 0.86921818 0.40402358 0.58494053 0.51427792 0.37182423 0.38153149
 0.78325317 0.88029932 0.54753919 0.68413757 0.5528958  0.46608254
 0.17714669 0.81303191 0.70188628 0.83066482 0.41142305 0.82042243
 0.67756911 0.43774107 0.25514298 0.38148202 0.26362372 0.5330824
 0.55445361 0.36129619 0.45209885 0.69437544 0.196665   0.29929203
 0.23499873 0.29089935 0.23980452 0.58573239 0.78389179 0.67677784
 0.49441642 0.65158696 0.7594956  0.17764853 0.24953486 0.67033428
 0.63383012 0.28769443 0.5389995  0.25195742 0.12739347 0.6835986
 0.50172234 0.43357378 0.44851119 0.37269733 0.28081481 0.8260502
 0.58907161 0.64310683 0.80170182 0.3854436  0.50577893 0.71046629
 0.71094615 0.37523372 0.14596202 0.5834

<p style="font-size:18px;">Get Fitness Value</p>
The final fitness represents the performance of the ANN after its weights and biases have been optimised by the PSO

In [59]:
# create a dummy particle and assign the optimised parameters
optimised_particle = Particle(vectorSize)
optimised_particle.particlePosition = best_parameters

# convert the optimised particle position back to ANN weights and biases
optimised_ann = particleToAnn(
    optimised_particle,
    annLayers,
    activation_functions
)

# run through ANN with the optimised parameters
fitness_value = assessFitness(
    optimised_particle,  # pass the optimised particle for fitness evaluation
    dataset = (X_train_scaled, y_train),
    annLayers = annLayers,
    activationFunctions = activation_functions,
    loss_function = MeanSquaredError()
)
print("Fitness Value:", fitness_value)

Fitness Value: 754.1743507985171


<p style="font-size:18px;">Training the ANN</p>

In [60]:
y_train_pred = optimised_ann.forwardPropagation(X_train_scaled)
loss_function = MeanSquaredError()
training_loss = loss_function.evaluate(y_train_pred, y_train)

print(f"Training Loss: {training_loss}")

Training Loss: 754.1743507985171


<p style="font-size:18px;">Testing the model on the test set</p>

In [61]:
X_test_scaled = np.random.rand(100, 8)  # Replace with actual test data
y_test = np.random.rand(100, 1)  # Replace with actual test labels

y_test_pred = optimised_ann.forwardPropagation(X_test_scaled)
test_loss = loss_function.evaluate(y_test_pred, y_test)

print(f"Test Loss: {test_loss}")

Test Loss: 754.9299302793769


In [62]:
epochs = 500  # Set this to the number of epochs you want
max_iterations = 100
for epoch in range(epochs):
    y_train_pred = optimised_ann.forwardPropagation(X_train_scaled)

    loss_function = MeanSquaredError()
    training_loss = loss_function.evaluate(y_train_pred, y_train)

    # Print the training loss
    print(f"Epoch {epoch + 1}/{epochs} - Training Loss: {training_loss}")

    # Make predictions on the test set
    y_test_pred = optimised_ann.forwardPropagation(X_test_scaled)
    test_loss = loss_function.evaluate(y_test_pred, y_test)

    # Print the test loss
    print(f"Test Loss: {test_loss}")

Epoch 1/500 - Training Loss: 754.1743507985171
Test Loss: 754.9299302793769
Epoch 2/500 - Training Loss: 754.1743507985171
Test Loss: 754.9299302793769
Epoch 3/500 - Training Loss: 754.1743507985171
Test Loss: 754.9299302793769
Epoch 4/500 - Training Loss: 754.1743507985171
Test Loss: 754.9299302793769
Epoch 5/500 - Training Loss: 754.1743507985171
Test Loss: 754.9299302793769
Epoch 6/500 - Training Loss: 754.1743507985171
Test Loss: 754.9299302793769
Epoch 7/500 - Training Loss: 754.1743507985171
Test Loss: 754.9299302793769
Epoch 8/500 - Training Loss: 754.1743507985171
Test Loss: 754.9299302793769
Epoch 9/500 - Training Loss: 754.1743507985171
Test Loss: 754.9299302793769
Epoch 10/500 - Training Loss: 754.1743507985171
Test Loss: 754.9299302793769
Epoch 11/500 - Training Loss: 754.1743507985171
Test Loss: 754.9299302793769
Epoch 12/500 - Training Loss: 754.1743507985171
Test Loss: 754.9299302793769
Epoch 13/500 - Training Loss: 754.1743507985171
Test Loss: 754.9299302793769
Epoch 14